## Setup

If you do not have access to a nvidia GPU, install dependencies here:

In [ ]:
!pip install ctransformers

Run this in Google Colab/Sagemaker studio or if you are running locally and have access to a GPU:

In [ ]:
!pip uninstall ctransformers -y
!CT_CUBLAS=1 pip install ctransformers --no-binary ctransformers

Run this on Apple silicon (M1, M2):

In [ ]:
!pip uninstall ctransformers -y
!CT_METAL=1 pip install ctransformers --no-binary ctransformers

## Smoke test

**Smoke test**: Check that the model is loaded successfully and generates good-quality responses

In [ ]:
from ctransformers import AutoModelForCausalLM

llm = AutoModelForCausalLM.from_pretrained(
      'TheBloke/Llama-2-7B-Chat-GGML',
      model_file='llama-2-7b-chat.ggmlv3.q4_0.bin',
      model_type="llama",
      #lib='avx2', for cpu use
      gpu_layers=110, #110 for 7b, 130 for 13b,
      context_length=4098,
      reset=True,
      threads=8,
      top_k=20,
      top_p=0.95,
      max_new_tokens=1000,
      repetition_penalty=1.1,
      temperature=0.1,
      stream=True,
      )

In [ ]:
def prompt(query, stream=True):
  if stream:
    response = ""
    for text in llm(query, stream=True):
      print(text, end="", flush=True)
      response += text
    return response
  else:
    return llm(query, stream=False)


In [ ]:
def template(query): return f"""
### Instruction: {query}. Be succinct, and return response as a 5-point list.
### Response:"""

prompt(template(query="How can I be a more balanced human being?"))

## Exercise 1: Manual exploratory testing

In [ ]:
RESUME_1 = """
Objective: Dedicated IT Developer with over 5 years of experience in full-stack
web development, mobile application development, and cloud computing. Seeking
to leverage my technical expertise and problem-solving skills to contribute to
a forward-thinking team at WidgetCraft.

Technical Skills:
- Languages: Java, Python, JavaScript, C#
- Web: HTML5, CSS3, Bootstrap, React, Angular, Node.js
- Mobile: Android (Java, Kotlin), iOS (Swift)
"""

RESUME_2 = """
Jane Doe, Marketing Manager

Professional Experience:

Marketing Manager | Horizon Marketing Solutions | 2018 - Present
- Spearheaded end-to-end marketing efforts for a diverse portfolio of clients,
resulting in a 20 percent average increase in annual revenue.
- Led a cross-functional team of 10 professionals, fostering a collaborative
environment and achieving a 30 percent improvement in campaign efficiency.
- Developed and executed data-driven marketing strategies, resulting in a 25
percent boost in online conversions and a 15 percent increase in social media
engagement.
- Conducted in-depth market analysis, competitor assessments, and consumer
behavior studies to identify new opportunities and optimize existing campaigns.
- Collaborated closely with clients to align marketing strategies with their
business goals, resulting in a 95 percent client retention rate.

Skills:
- Marketing Strategy, Brand Development, Campaign Management, Team Leadership,
Market Research, Digital Marketing, Data Analysis, Cross-functional Collaboration

"""

In [ ]:
def extract_skills(resume): return f"""
### Instruction: Extract technical skills from this document:
{resume}
### Response:"""

result = prompt(extract_skills(RESUME_1))

The result is human-readable, but not a very helpful data structure. Let's get the model to return JSON so we can work with it in automated tests (Exercise 2) and any other downstream components.

In [ ]:
def extract_json(resume): return f"""
### Instruction: Extract technical skills from this document using only
information in the following document. Return results in valid JSON format.
{resume}
### Response:
"""

response = prompt(extract_json(RESUME_1), stream=True)

## Exercise 2: Automated tests. Example-based tests

In [ ]:
import json

def extract_json2(resume): return """
### Instruction: Extract technical skills from this document using only
information in the following document.
Return results in valid JSON format.
Final answer should be in the following format:

{
  {"technical_skills":
   {
      {
          "key_1": ["value1", "value2", "value3" ],
          "key_2": ["value1", "value2", "value3"]
      }
    }
   }
}

Ensure that key_1, key_2, etc. are exact valid keys from user input,
presented in snake_case""" + f"""

### Input: {resume}

### Response:"""

response = prompt(extract_json2(RESUME_1))

In [ ]:
actual_skills = json.loads(response)

expected_skills = {
    "technical_skills": {
        "languages": ["Java", "Python", "JavaScript", "C#"],
        "web": ["HTML5", "CSS3", "Bootstrap", "React", "Angular", "Node.js"],
        "mobile": ["Android (Java, Kotlin)", "iOS (Swift)"],
    }
}

print(f"Actual:   {actual_skills}")
print(f"Expected: {expected_skills}")
assert actual_skills == expected_skills

The failing test in the preceding cell, though jarring, is a good thing! It helped us catch a bug: There was some information loss as some skills were not included in the result. Let's try to get this test to pass with better prompting

In [ ]:
import json

def extract_json3(resume): return """
[INST] <<SYS>>
You are an assistant responsible for extracting attributes from unstructured
text and returning them as a valid JSON.
In your response, DO NOT include any text other than the JSON response.
Keys and values should be quoted with "". No trailing commas.

Example: This is a resume, and I'm listing my technical skills:
- Fruits: Apple, Banana, Coconut
- Raw Vegetables: Lettuce, Cabbage, Zucchini, Carrots (Diced, Sliced)

The response should then be in the format:
{
    "technical_skills":
      {
        {
            "fruits": ["Apple", "Banana", "Coconut"],
            "raw_vegetables": ["Lettuce", "Cabbage", "Zucchini", "Carrots (Diced, Sliced)"]
        }
    }
}

<</SYS>>"""+f"""

Extract technical skills from the given document using only information in
the following document:

{resume}

Only include the JSON response.
[/INST]
"""
response = prompt(extract_json3(RESUME_1))

In [ ]:
actual_skills = json.loads(response)
expected_skills = {
    "technical_skills": {
        "languages": ["Java", "Python", "JavaScript", "C#"],
        "web": ["HTML5", "CSS3", "Bootstrap", "React", "Angular", "Node.js"],
        "mobile": ["Android (Java, Kotlin)", "iOS (Swift)"],
    }
}

print(f"Actual:   {actual_skills}")
print(f"Expected: {expected_skills}")
assert actual_skills == expected_skills

## Exercise 3: Using an LLM to evaluate itself (or another LLM)

In [ ]:
def summarizer(resume): return f"""
[INST]
<<SYS>>
You are a helpful assistant, skilled at providing succinct and accurate
summaries of an applicant based on their resume. In your response, include only
the summary no preamble.
<</SYS>>

Instruction: Generate a two-sentence summary of:

{resume}

[/INST]

"""
summary1 = prompt(summarizer(RESUME_1))

In [ ]:
def summary_evaluator(resume, summary): return f"""
<SYS>
You are a strict evaluator responsible for checking if summaries are accurate
or not.
</SYS>
[INST]
<Resume>
{resume}
</Resume>

<Summary>
{summary}
</Summary>

Instruction: Evaluate if <Summary> is an accurate summary of <Resume>. Be critical o
Present response in a JSON format with keys of score 1-10 and a short reason.
[/INST]
"""

evaluation = prompt(summary_evaluator(RESUME_1, summary1))

In [ ]:
summary_2 = prompt(summarizer(RESUME_2))
evaluation2 = prompt(summary_evaluator(RESUME_2, summary_2))
print(evaluation2)

In [ ]:
evaluation3 = prompt(summary_evaluator(RESUME_2, "Bob Dole is a zookeeper"))
print(evaluation3)